In [ ]:
%pip install -U langchain-ollama

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental

In [ ]:
import pandas as pd
from genscai import paths
from langchain_core.documents import Document

df_modeling_papers = pd.read_json(paths.data / "modeling_papers_0.json", orient="records", lines=True)

documents = []

for row in df_modeling_papers.itertuples():
    documents.append(Document(id=row.id, page_content=row.abstract))

print(len(documents))

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

# from langchain_openai import ChatOpenAI
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1")

llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
        "Disease Model",
        "Diesase Model Type",
        "Disease",
        "Geographic Location",
        "Disease Vector",
        "Infected Population",
    ],
    allowed_relationships=[
        "MODEL_APPLIED_TO_DISEASE",
        "MODEL_HAS_TYPE",
        "DISEASE_LOCATED_IN",
        "INFECTED_BY",
        "DISEASE_PRESENT_IN",
        "DISEASE_CARRIED_BY",
    ],
)

graph_documents = llm_transformer_filtered.convert_to_graph_documents(documents[:10])

In [ ]:
for doc in graph_documents:
    if len(doc.nodes) > 0:
        print(f"Paper ID: {doc.source.id}")
        print(f"Paper Abstract: {doc.source.page_content}")

        for node in doc.nodes:
            print(f"Node: {node.id}, Type: {node.type}")
        for rel in doc.relationships:
            print(f"Relationship: {rel.type}")
            print(f"   Source: {rel.source.id}, Type: {rel.source.type}")
            print(f"   Target: {rel.target.id}, Type: {rel.target.type}")

        print()